In [1]:
# importa os pacotes necessários
import numpy as np
import os, cv2, random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import np_utils
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import  cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from skimage.feature import greycomatrix, greycoprops
from skimage.feature import hog

In [2]:
#funções de leitura e preparação das imagens
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype=np.uint8)

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image.T
        if i%250 == 0: print('Processed {} of {}'.format(i, count))    
    return data

def show_cats_and_dogs(idx):
    cat = read_image(train_cats[idx])
    dog = read_image(train_dogs[idx])
    pair = np.concatenate((cat, dog), axis=1)
    plt.figure(figsize=(10,5))
    plt.imshow(pair)
    plt.show()
    
# dois exemplos de descritores. Você deve criar outros mais robustos.
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):     
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    #image = cv2.imread(image_file)        
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
        [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    # return the flattened histogram as the feature vector
    return hist.flatten()

In [3]:
TRAIN_DIR = 'kaggle/cifar-10/train'
TRAIN_LABELS_DIR = 'kaggle/cifar-10/trainLabels.csv'

ROWS = 128
COLS = 128
CHANNELS = 3
NIM = 1000

image_paths = os.listdir(TRAIN_DIR)

train_images = [read_image(TRAIN_DIR + "/" + i) for i in image_paths]
train_labels = pd.read_csv(TRAIN_LABELS_DIR)

#num_classes = 10
#y_train = np_utils.to_categorical(y_train,num_classes)
#y_test = np_utils.to_categorical(y_test,num_classes)

# considera apenas NIM imagens. Para o dataset completo, desconsiderar.
# random.shuffle(train_images)

In [4]:
rawImages = []
descHist = []

count = len(train_images)

for i, image in enumerate(train_images):
    pixels = image_to_feature_vector(image)
    histogram = extract_color_histogram(image)
    
    rawImages.append(pixels)
    descHist.append(histogram)
        
    if i%1000 == 0: print('Processed {} of {}'.format(i, count))

Processed 0 of 50000
Processed 1000 of 50000
Processed 2000 of 50000
Processed 3000 of 50000
Processed 4000 of 50000
Processed 5000 of 50000
Processed 6000 of 50000
Processed 7000 of 50000
Processed 8000 of 50000
Processed 9000 of 50000
Processed 10000 of 50000
Processed 11000 of 50000
Processed 12000 of 50000
Processed 13000 of 50000
Processed 14000 of 50000
Processed 15000 of 50000
Processed 16000 of 50000
Processed 17000 of 50000
Processed 18000 of 50000
Processed 19000 of 50000
Processed 20000 of 50000
Processed 21000 of 50000
Processed 22000 of 50000
Processed 23000 of 50000
Processed 24000 of 50000
Processed 25000 of 50000
Processed 26000 of 50000
Processed 27000 of 50000
Processed 28000 of 50000
Processed 29000 of 50000
Processed 30000 of 50000
Processed 31000 of 50000
Processed 32000 of 50000
Processed 33000 of 50000
Processed 34000 of 50000
Processed 35000 of 50000
Processed 36000 of 50000
Processed 37000 of 50000
Processed 38000 of 50000
Processed 39000 of 50000
Processed 400

In [5]:
labels = list(train_labels.label.unique())
from sklearn.preprocessing import LabelEncoder

def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df2 = pd.concat([df, dummies], axis=1)
    return df2

df2 = one_hot(train_labels,['label'])
del df2['label']
del df2['id']

print(df2.columns)


Index(['label_airplane', 'label_automobile', 'label_bird', 'label_cat',
       'label_deer', 'label_dog', 'label_frog', 'label_horse', 'label_ship',
       'label_truck'],
      dtype='object')


In [ ]:
#Avalia o primeiro descritor: as imagens raw

(X_train, X_test, y_train, y_test) = train_test_split(rawImages, df2, test_size=0.10, random_state=42)

classifiers = [
    KNeighborsClassifier(17),    
    DecisionTreeClassifier(),
    GaussianNB()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    print("did")
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))    

KNeighborsClassifier
****Results****
did


In [ ]:
#Avalia o segundo descritor: color histogram

(X_train, X_test, y_train, y_test) = train_test_split(descHist, labels, test_size=0.25, random_state=42)
classifiers = [KNeighborsClassifier(17), DecisionTreeClassifier(), GaussianNB()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))   

In [ ]:
#Avalia a combinação dos dois primeiros descritores!

#ATENÇÃO: ESTE É APENAS UM CÓDIGO EXEMPLO. VOCÊ DEVE DESENVOLVER
#DESCRITORES MAIS ROBUSTOS, BEM COMO EXPLORAR MELHOR AS MÉTRICAS
#DE AVALIAÇÃO (MATRIZ DE CONFUSÃO, ETC)

trainAux = np.hstack((descHist, rawImages))
(X_train, X_test, y_train, y_test) = train_test_split(trainAux, labels, test_size=0.25, random_state=42)

classifiers = [
    KNeighborsClassifier(17),    
    DecisionTreeClassifier(),
    GaussianNB()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    print("accuracy: {:.2f}%".format(acc * 100))   